In [6]:
import pandas as pd
from helpers import (
    get_user_genre_list,
    get_user_max_likelihood,
    get_ideal_rankings,
    get_all_genres_list
)

test_data = pd.read_csv('output/test.csv')
ratings = pd.read_csv('datasets/ml-latest-small/ratings.csv')
movies = pd.read_csv('datasets/ml-latest-small/movies.csv')
rankings = pd.read_csv('output/rankings.csv')

# merge item data to the ratings df
test_data_full = test_data.merge(movies, how='inner', on='movieId').sort_values(by='userId')
ratings_full = ratings.merge(movies, how='inner', on='movieId').sort_values(by='userId')
rankings_full = rankings.merge(movies, how='inner', on='movieId').sort_values(by='rank')

# create a new column for the genres count
test_data_full['count_genres'] = test_data_full.apply(lambda x: len(x['genres'].split('|')), axis=1)

In [2]:
test_data_full[test_data_full['userId'] == 2]

,userId,movieId,rating,title,genres,count_genres
18173,2,89774,5.0,Warrior (2011),Drama,1
14240,2,112552,4.0,Whiplash (2014),Drama,1
6468,2,48516,4.0,"Departed, The (2006)",Crime|Drama|Thriller,3
20210,2,99114,3.5,Django Unchained (2012),Action|Drama|Western,3


In [3]:
user_id = 2
k = 4

top_k_ideal = get_ideal_rankings(user_id, test_data_full, k)

In [4]:
top_k_ideal

[(2, 89774, 5.0), (2, 48516, 4.0), (2, 112552, 4.0), (2, 99114, 3.5)]

In [10]:
genres = get_all_genres_list(ratings_full)
genres

['Action',
 'Mystery',
 'Children',
 'Fantasy',
 'Drama',
 'Horror',
 'Musical',
 'Adventure',
 'Comedy',
 'Sci-Fi',
 'Western',
 'War',
 'Film-Noir',
 'Documentary',
 'Animation',
 'Romance',
 'Thriller',
 'IMAX',
 'Crime']

In [11]:
gamma = get_user_max_likelihood(user_id, 'Action', test_data_full)
gamma

0.1111111111111111

In [ ]:
rankings_full[rankings_full['userId'] == 2]
